In [1]:
number_of_center = [[21,16,11,22,6],
                    [31,17,16,11,3],
                    [23,16,12,13,4],
                    [17,11,21,16,2],
                    [20,11,21,16,2]]

state_max_cap = [5000,10000,7500,8500,9500]

state_population_c_b_a = [[115900,434890,15000],
                          [100450,378860,35234],
                          [223400,643320,22318],
                          [269300,859900,23893],
                          [221100,450500,19284]]

In [2]:
import constraint,math

def grade(vaccine_per_day,max_cap):
    return abs(vaccine_per_day-max_cap)

for i in range(5):
    print('ST-'+str(i+1))
    print()
    
    problem = constraint.Problem()

    # number of center
    problem.addVariable('CR-1', range(number_of_center[i][0]))  
    problem.addVariable('CR-2', range(number_of_center[i][1]))
    problem.addVariable('CR-3', range(number_of_center[i][2]))
    problem.addVariable('CR-4', range(number_of_center[i][3]))
    problem.addVariable('CR-5', range(number_of_center[i][4]))

    # check whether the solutions exceed the maximum vaccination capacity of the state
    def check_state_max_cap(a, b, c, d, e):  
        if (a*200 + b*500 + c*1000 + d*2500 + e*4000) <= state_max_cap[i]:
            return True

    # add the constraint
    problem.addConstraint(check_state_max_cap,['CR-1','CR-2','CR-3','CR-4','CR-5'])

    #####--------------------------------- Normal Day ---------------------------------------#####
    minimum_rental_normal = float('inf')
    dif = float('inf')
    solution_found_normal = {}
    total_day = 0
    solutions = problem.getSolutions()

    for s in solutions:
        vaccine_per_day = s['CR-1']*200 + s['CR-2']*500 + s['CR-3']*1000 + s['CR-4']*2500 + s['CR-5']*4000

        if (vaccine_per_day != 0 and dif >= grade(vaccine_per_day,state_max_cap[i])): #reach max capacity from all the solutions
            current_rental_per_day = s['CR-1']*100+s['CR-2']*250+s['CR-3']*500+s['CR-4']*800+s['CR-5']*1200

            # find the minimum rental
            if current_rental_per_day < minimum_rental_normal:
                dif = grade(vaccine_per_day,state_max_cap[i])
                minimum_rental_normal = current_rental_per_day
                solution_found_normal = s

                total_day = math.floor(sum(state_population_c_b_a[i]) / vaccine_per_day)

    # vaccine given per day
    current_vaccine_given = total_day * (solution_found_normal['CR-1']*200 + solution_found_normal['CR-2']*500 + solution_found_normal['CR-3']*1000 + solution_found_normal['CR-4']*2500 + solution_found_normal['CR-5']*4000)
    # remaining population for the last day
    remaining = sum(state_population_c_b_a[i])-current_vaccine_given

    #####--------------------------------- Remaining ---------------------------------------#####
    minimum_rental_last = float('inf') 
    solution_found_last = {}

    for s in solutions:  
        vaccine_given = s['CR-1']*200 + s['CR-2']*500 + s['CR-3']*1000 + s['CR-4']*2500 + s['CR-5']*4000 #capacity

        if vaccine_given >= remaining:
            remaining_rental = s['CR-1']*100+s['CR-2']*250+s['CR-3']*500+s['CR-4']*800+s['CR-5']*1200

            if remaining_rental < minimum_rental_last:
                minimum_rental_last = remaining_rental
                solution_found_last = s

    print('Total population:',sum(state_population_c_b_a[i]))
    print('Vac_A needed:',state_population_c_b_a[i][2])
    print('Vac_B needed:',state_population_c_b_a[i][1])
    print('Vac_C needed:',state_population_c_b_a[i][0])
    print('Total day needed:',total_day+1,'days')
    print('Total rental: RM{}*{} + RM{}*1 = RM{}'.format(minimum_rental_normal,total_day,minimum_rental_last,minimum_rental_normal*total_day+minimum_rental_last))    
    print('-------------------------------------------------------------------------------------------')
    print('Normal day solution:',solution_found_normal)
    vac_per_normal_day = solution_found_normal['CR-1']*200 + solution_found_normal['CR-2']*500 + solution_found_normal['CR-3']*1000 + solution_found_normal['CR-4']*2500 + solution_found_normal['CR-5']*4000
    vac_A = math.floor(vac_per_normal_day*state_population_c_b_a[i][2]/sum(state_population_c_b_a[i]))
    vac_B = math.floor(vac_per_normal_day*state_population_c_b_a[i][1]/sum(state_population_c_b_a[i]))
    vac_C = vac_per_normal_day - vac_A - vac_B
    print('Normal day vaccination per day:',vac_per_normal_day)
    print('Vac_A per day:',vac_A)
    print('Vac_B per day:',vac_B)
    print('Vac_C per day:',vac_C)
    print('Normal day total vaccination coverage:',(solution_found_normal['CR-1']*200 + solution_found_normal['CR-2']*500 + solution_found_normal['CR-3']*1000 + solution_found_normal['CR-4']*2500 + solution_found_normal['CR-5']*4000)*total_day)
    print('-------------------------------------------------------------------------------------------')
    print('Last day solution:',solution_found_last)
    vac_A = state_population_c_b_a[i][2] - (vac_A*total_day)
    vac_B = state_population_c_b_a[i][1] - (vac_B*total_day)
    vac_C = state_population_c_b_a[i][0] - (vac_C*total_day)
    print('LAst day vaccination:',remaining)
    print('Vac_A last day:',vac_A)
    print('Vac_B last day:',vac_B)
    print('Vac_C last day:',vac_C)
    print('Last day vaccination coverage:',solution_found_last['CR-1']*200 + solution_found_last['CR-2']*500 + solution_found_last['CR-3']*1000 + solution_found_last['CR-4']*2500 + solution_found_last['CR-5']*4000)
    print('===========================================================================================')
    print()

ST-1

Total population: 565790
Vac_A needed: 15000
Vac_B needed: 434890
Vac_C needed: 115900
Total day needed: 114 days
Total rental: RM1600*113 + RM400*1 = RM181200
-------------------------------------------------------------------------------------------
Normal day solution: {'CR-5': 0, 'CR-3': 0, 'CR-2': 0, 'CR-1': 0, 'CR-4': 2}
Normal day vaccination per day: 5000
Vac_A per day: 132
Vac_B per day: 3843
Vac_C per day: 1025
Normal day total vaccination coverage: 565000
-------------------------------------------------------------------------------------------
Last day solution: {'CR-5': 0, 'CR-3': 0, 'CR-2': 0, 'CR-1': 4, 'CR-4': 0}
LAst day vaccination: 790
Vac_A last day: 84
Vac_B last day: 631
Vac_C last day: 75
Last day vaccination coverage: 800

ST-2

Total population: 514544
Vac_A needed: 35234
Vac_B needed: 378860
Vac_C needed: 100450
Total day needed: 52 days
Total rental: RM3200*51 + RM1500*1 = RM164700
-----------------------------------------------------------------------